In [1]:
# 附加题中，我们选择使用cifar10
import paddle
from paddle.vision.transforms import Normalize, Compose, Transpose
from paddle.vision.datasets import Cifar10
def get_cifar10_dataloader():
    # cifar10是一个三通道的图片
    transforms = Compose([Normalize(mean=[127.5, 127.5, 127.5], std= [127.5, 127.5, 127.5], data_format='HWC'), Transpose() ])
    # 数据集
    train_datasets = Cifar10(mode='train', transform=transforms)
    test_datasets = Cifar10(mode='test', transform=transforms)
    # 数据读取器
    train_loader = paddle.io.DataLoader(train_datasets, batch_size=64, shuffle = True, num_workers=1, drop_last=True)
    test_loader = paddle.io.DataLoader(test_datasets, batch_size=64, shuffle = False, num_workers=1, drop_last=False)
    return train_loader, test_loader

In [2]:
train_loader, test_loader = get_cifar10_dataloader()

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\io\reader.py:433: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [3]:
import visualdl
import visualdl.server.app
visualdl.server.app.run('./run/cifar_log',
                        host = "127.0.0.1",
                        port=8081,
                        cache_timeout=5)

18896

In [4]:
# 定义模型结构
# 首先只来一层卷积

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_ONE_CONV(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_ONE_CONV, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=5120, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [5]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/one_conv")
def train_Cifar_ONE(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 10
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [6]:
model_cifar_one = Cifar_ONE_CONV()
train_Cifar_ONE(model_cifar_one)

epoch: 0, batch: 0, loss is: [2.7267888], acc is [0.078125]
epoch: 0, batch: 150, loss is: [1.4831171], acc is [0.5]
epoch: 0, batch: 300, loss is: [1.4324566], acc is [0.578125]
epoch: 0, batch: 450, loss is: [1.242635], acc is [0.5625]
epoch: 0, batch: 600, loss is: [1.7445328], acc is [0.40625]
epoch: 0, batch: 750, loss is: [1.2230357], acc is [0.578125]
epoch: 1, batch: 0, loss is: [1.6908634], acc is [0.375]
epoch: 1, batch: 150, loss is: [1.2385381], acc is [0.515625]
epoch: 1, batch: 300, loss is: [1.397466], acc is [0.53125]
epoch: 1, batch: 450, loss is: [1.1587546], acc is [0.59375]
epoch: 1, batch: 600, loss is: [1.4375345], acc is [0.5625]
epoch: 1, batch: 750, loss is: [1.5021424], acc is [0.484375]
epoch: 2, batch: 0, loss is: [1.4640067], acc is [0.46875]
epoch: 2, batch: 150, loss is: [1.1618476], acc is [0.546875]
epoch: 2, batch: 300, loss is: [1.7091419], acc is [0.5]
epoch: 2, batch: 450, loss is: [1.4363967], acc is [0.5625]
epoch: 2, batch: 600, loss is: [1.36101

In [10]:
# 定义模型结构

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_TWO_CONV(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=1536, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [11]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_2")
def train_Cifar_TWO(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 10  
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [12]:
model_cifar_two = Cifar_TWO_CONV()
train_Cifar_TWO(model_cifar_two)

epoch: 0, batch: 0, loss is: [3.0159125], acc is [0.09375]
epoch: 0, batch: 150, loss is: [1.7808365], acc is [0.40625]
epoch: 0, batch: 300, loss is: [1.6551902], acc is [0.46875]
epoch: 0, batch: 450, loss is: [1.2367995], acc is [0.578125]
epoch: 0, batch: 600, loss is: [1.5560794], acc is [0.4375]
epoch: 0, batch: 750, loss is: [1.3427678], acc is [0.46875]
epoch: 1, batch: 0, loss is: [1.4758031], acc is [0.53125]
epoch: 1, batch: 150, loss is: [1.3802402], acc is [0.546875]
epoch: 1, batch: 300, loss is: [1.1563023], acc is [0.5625]
epoch: 1, batch: 450, loss is: [1.570127], acc is [0.421875]
epoch: 1, batch: 600, loss is: [1.2995212], acc is [0.5625]
epoch: 1, batch: 750, loss is: [1.0789251], acc is [0.546875]
epoch: 2, batch: 0, loss is: [1.2229929], acc is [0.53125]
epoch: 2, batch: 150, loss is: [1.3716687], acc is [0.484375]
epoch: 2, batch: 300, loss is: [1.4375595], acc is [0.46875]
epoch: 2, batch: 450, loss is: [1.2701385], acc is [0.578125]
epoch: 2, batch: 600, loss i

In [42]:
# 定义模型结构
# 首先只来一层卷积

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_THree_CONV2(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_THree_CONV2, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再定义一层卷积层
        self.conv3 = Conv2D(in_channels=20, out_channels=32,kernel_size=3, stride=1, padding=1)
        self.max_pool3 = MaxPool2D(kernel_size=2,stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=2048, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [43]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/three_conv2")
def train_Cifar_Three(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 15
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [44]:
model_cifar_two2 = Cifar_THree_CONV2()
train_Cifar_Three(model_cifar_two2)


epoch: 0, batch: 0, loss is: [2.5732203], acc is [0.09375]
epoch: 0, batch: 150, loss is: [1.7769077], acc is [0.3125]
epoch: 0, batch: 300, loss is: [1.5719324], acc is [0.484375]
epoch: 0, batch: 450, loss is: [1.6376365], acc is [0.375]
epoch: 0, batch: 600, loss is: [1.1803364], acc is [0.625]
epoch: 0, batch: 750, loss is: [1.8372405], acc is [0.28125]
epoch: 1, batch: 0, loss is: [1.2950017], acc is [0.5625]
epoch: 1, batch: 150, loss is: [1.9494592], acc is [0.375]
epoch: 1, batch: 300, loss is: [1.370954], acc is [0.46875]
epoch: 1, batch: 450, loss is: [1.6023482], acc is [0.390625]
epoch: 1, batch: 600, loss is: [1.2391276], acc is [0.515625]
epoch: 1, batch: 750, loss is: [1.3884648], acc is [0.46875]
epoch: 2, batch: 0, loss is: [1.0867006], acc is [0.65625]
epoch: 2, batch: 150, loss is: [1.3110088], acc is [0.484375]
epoch: 2, batch: 300, loss is: [1.4036814], acc is [0.453125]
epoch: 2, batch: 450, loss is: [1.3103821], acc is [0.5625]
epoch: 2, batch: 600, loss is: [1.3

In [20]:
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_TWO_CONV2_drop(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_drop, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再定义一层卷积层
        self.conv3 = Conv2D(in_channels=20, out_channels=32,kernel_size=5, stride=1, padding=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=2048, out_features=3072)
        self.fc2 = Linear(in_features=3072,out_features=1024)
        self.fc3 = Linear(in_features=1024, out_features=10)
    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)
        x = F.dropout(x, p = 0.007,mode=mode)
        x = self.fc2(x)
        x = F.dropout(x, p = 0.005, mode=mode)
        x = self.fc3(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [22]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
logwriter = LogWriter(logdir="./run/cifar_log/three_conv_drop")
def train_Cifar_TWO2_drop(model):
    # 定义优化器
    EPOCH_NUM = 15
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels,mode='downscale_in_infer')
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [23]:
model_cifar_drop = Cifar_TWO_CONV2_drop()
train_Cifar_TWO2_drop(model_cifar_drop)

epoch: 0, batch: 0, loss is: [3.6741738], acc is [0.03125]
epoch: 0, batch: 150, loss is: [2.6778302], acc is [0.09375]
epoch: 0, batch: 300, loss is: [2.422026], acc is [0.09375]
epoch: 0, batch: 450, loss is: [2.3471777], acc is [0.078125]
epoch: 0, batch: 600, loss is: [2.3338192], acc is [0.0625]
epoch: 0, batch: 750, loss is: [2.3259423], acc is [0.140625]
epoch: 1, batch: 0, loss is: [2.3339372], acc is [0.09375]
epoch: 1, batch: 150, loss is: [2.3465264], acc is [0.125]
epoch: 1, batch: 300, loss is: [2.3495932], acc is [0.125]
epoch: 1, batch: 450, loss is: [2.2595706], acc is [0.1875]
epoch: 1, batch: 600, loss is: [2.3430743], acc is [0.109375]
epoch: 1, batch: 750, loss is: [2.339295], acc is [0.09375]
epoch: 2, batch: 0, loss is: [2.298511], acc is [0.125]
epoch: 2, batch: 150, loss is: [2.3004704], acc is [0.140625]
epoch: 2, batch: 300, loss is: [4908.909], acc is [0.125]
epoch: 2, batch: 450, loss is: [101.91498], acc is [0.078125]
epoch: 2, batch: 600, loss is: [4.69847

In [39]:
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_norm(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_norm, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(12)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(24)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第三个卷积层
        self.conv3 = Conv2D(in_channels=24, out_channels= 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(32)
        self.max_pool3 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是32*8*8,定义一个全连接
        self.fc = Linear(in_features=512, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [40]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_new_batnorm1")
def train_Cifar_TWO2_norm(model):
    # 定义优化器
    EPOCH_NUM = 15
    opt = paddle.optimizer.Adam(learning_rate=0.01, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [41]:
model = Cifar_TWO_CONV2_norm()
train_Cifar_TWO2_norm(model)

epoch: 0, batch: 0, loss is: [2.970825], acc is [0.234375]
epoch: 0, batch: 150, loss is: [1.5816582], acc is [0.359375]
epoch: 0, batch: 300, loss is: [1.4609181], acc is [0.4375]
epoch: 0, batch: 450, loss is: [1.1803728], acc is [0.59375]
epoch: 0, batch: 600, loss is: [0.94525087], acc is [0.65625]
epoch: 0, batch: 750, loss is: [1.3759532], acc is [0.578125]
epoch: 1, batch: 0, loss is: [1.1294346], acc is [0.640625]
epoch: 1, batch: 150, loss is: [1.0835528], acc is [0.609375]
epoch: 1, batch: 300, loss is: [0.7557789], acc is [0.75]
epoch: 1, batch: 450, loss is: [1.0806919], acc is [0.59375]
epoch: 1, batch: 600, loss is: [1.0914581], acc is [0.6875]
epoch: 1, batch: 750, loss is: [0.90149915], acc is [0.71875]
epoch: 2, batch: 0, loss is: [1.011965], acc is [0.671875]
epoch: 2, batch: 150, loss is: [1.1985189], acc is [0.640625]
epoch: 2, batch: 300, loss is: [0.9783133], acc is [0.6875]
epoch: 2, batch: 450, loss is: [1.025254], acc is [0.671875]
epoch: 2, batch: 600, loss is

In [45]:
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_norm_end(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_norm_end, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(12)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(24)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第三个卷积层
        self.conv3 = Conv2D(in_channels=24, out_channels= 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(32)
        self.max_pool3 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是32*8*8,定义一个全连接
        self.fc = Linear(in_features=512, out_features=218)
        self.fc2 = Linear(in_features=218,out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)
        x = F.dropout(x)
        x = self.fc2(x)

        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [47]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_end")
def train_Cifar_TWO2_end(model):
    # 定义优化器
    EPOCH_NUM = 20
    opt = paddle.optimizer.Adam(learning_rate=0.01, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100

                

In [48]:
mdoel = Cifar_TWO_CONV2_norm_end()
train_Cifar_TWO2_end(model)

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\nn\layer\norm.py:777: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


epoch: 0, batch: 0, loss is: [0.64566314], acc is [0.75]
epoch: 0, batch: 150, loss is: [0.48755217], acc is [0.859375]
epoch: 0, batch: 300, loss is: [0.5207561], acc is [0.8125]
epoch: 0, batch: 450, loss is: [0.8076332], acc is [0.75]
epoch: 0, batch: 600, loss is: [0.7619858], acc is [0.703125]
epoch: 0, batch: 750, loss is: [0.65657187], acc is [0.71875]
epoch: 1, batch: 0, loss is: [0.44764915], acc is [0.859375]
epoch: 1, batch: 150, loss is: [0.5023011], acc is [0.828125]
epoch: 1, batch: 300, loss is: [0.5162792], acc is [0.765625]
epoch: 1, batch: 450, loss is: [0.49148616], acc is [0.859375]
epoch: 1, batch: 600, loss is: [0.584095], acc is [0.828125]
epoch: 1, batch: 750, loss is: [0.54790777], acc is [0.8125]
epoch: 2, batch: 0, loss is: [0.70908505], acc is [0.703125]
epoch: 2, batch: 150, loss is: [0.43550918], acc is [0.859375]
epoch: 2, batch: 300, loss is: [0.74243844], acc is [0.796875]
epoch: 2, batch: 450, loss is: [0.52732307], acc is [0.765625]
epoch: 2, batch: 6

: 

In [41]:
# 定义模型结构
# 把以上的都综合在一起

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_lenet_3(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_lenet_3, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(32)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(64)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再增加一层卷积
        self.conv3 = Conv2D(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(64)

        # 这样输出的就是20*16*16,定义一个全连接
        self.fc1 = Linear(in_features=4096, out_features=10)
        self.fc3 = Linear(in_features=1024, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = paddle.to_tensor(x)
        x = self.fc1(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [42]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_new_lenet7")
def train_Cifar_TWO2_lenet(model):
    # 定义优化器
    EPOCH_NUM = 15
    opt = paddle.optimizer.Adam(learning_rate=0.005, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%200 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=200
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 200 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=200

                

In [43]:
model = Cifar_TWO_CONV2_lenet_3()
train_Cifar_TWO2_lenet(model)

epoch: 0, batch: 0, loss is: [2.5494406], acc is [0.1328125]
epoch: 0, batch: 200, loss is: [1.546255], acc is [0.4609375]
epoch: 1, batch: 0, loss is: [1.4955535], acc is [0.5]
epoch: 1, batch: 200, loss is: [1.3238297], acc is [0.6015625]
epoch: 2, batch: 0, loss is: [1.2607361], acc is [0.59375]
epoch: 2, batch: 200, loss is: [1.2540855], acc is [0.6328125]
epoch: 3, batch: 0, loss is: [1.102382], acc is [0.640625]
epoch: 3, batch: 200, loss is: [1.1954417], acc is [0.6640625]
epoch: 4, batch: 0, loss is: [1.124525], acc is [0.640625]
epoch: 4, batch: 200, loss is: [1.3655242], acc is [0.578125]
epoch: 5, batch: 0, loss is: [1.1353847], acc is [0.625]
epoch: 5, batch: 200, loss is: [1.190679], acc is [0.625]
epoch: 6, batch: 0, loss is: [1.0680808], acc is [0.609375]
epoch: 6, batch: 200, loss is: [1.2858982], acc is [0.6328125]
epoch: 7, batch: 0, loss is: [1.3763033], acc is [0.578125]
epoch: 7, batch: 200, loss is: [1.1359574], acc is [0.640625]
epoch: 8, batch: 0, loss is: [1.12